# Bài 1: thực hiện tạo recommendation system dựa trên content-based

## Functional Description
- Build recommendation system using content-based
- The system should recommend 10 most similar product names and description.
- The system should prioritize to recommend products in the same subcategory first if the subcategory could be determined.
- The system should prioritize to recommend products with higher rating.



## Use cases
- Use case 1: when user view a product, recommend 10 similar products.
- Use case 2: when user search for a product, recommend 10 results that most match with key words.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
import seaborn as sns

from utils import filter_vietnamese_words

In [3]:
# Load df from cleaned data

df = pd.read_pickle("./Products_ThoiTrangNam_cleaned.pkl")


In [4]:
df.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách T...
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu ...
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay á...
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83...,NaN,45000.0,4.8,Áo Ba Lỗ Chiều dài tay áo Không tay Phong c...


In [3]:
STOP_WORD_FILE = './vietnamese-stopwords.txt'
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [4]:
# filter Vietnamese words
df['description_clean'] = df['description'].apply(filter_vietnamese_words)


In [5]:
df.head()


,product_id,product_name,category,sub_category,link,image,price,rating,description,description_clean
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách T...,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu ...,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay á...,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi...,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi từ...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83...,NaN,45000.0,4.8,Áo Ba Lỗ Chiều dài tay áo Không tay Phong c...,Áo Ba Lỗ Chiều dài tay áo Không tay Phong cách...


In [6]:
# create the content
# as we keept the subcategories in the description, let only use product name and description to create the content

df['Content'] = df['product_name'] + ' ' + df['description_clean'].apply(lambda x: ' '.join(x.split()[:200]))

In [7]:
df.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description,description_clean,Content
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách T...,Áo Ba Lỗ Chiều dài tay áo Khác Phong cách Thể ...,"Áo ba lỗ thun gân ,form body tôn dáng Áo Ba Lỗ..."
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu ...,Áo Ba Lỗ Xuất xứ Việt Nam Tên tổ chức chịu trá...,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu ..."
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay á...,Áo Ba Lỗ Thương hiệu Tyasuo Chiều dài tay áo K...,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không..."
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi...,Áo Ba Lỗ Chất liệu Cotton Kho hàng 7981 Gửi từ...,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON Áo Ba Lỗ Ch...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83...,NaN,45000.0,4.8,Áo Ba Lỗ Chiều dài tay áo Không tay Phong c...,Áo Ba Lỗ Chiều dài tay áo Không tay Phong cách...,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...


In [8]:
#!pip install pandarallel
from pandarallel import pandarallel

In [9]:
# word_tokenize
# use to split sentences into words, combined words returned as word1_word2
# Example: >>> sentence = "Bác sĩ bây giờ có thể thản nhiên báo tin bệnh nhân bị ung thư"
        # >>> word_tokenize(sentence)
        # ["Bác sĩ", "bây giờ", "có thể", "thản nhiên", "báo tin", "bệnh nhân", "bị", "ung thư"]
pandarallel.initialize(progress_bar=True, nb_workers=4)



# df["Content_wt"]=df["Content"].apply(lambda x: word_tokenize(x, format="text"))

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [10]:
def tokenize_text(text):
    from underthesea import word_tokenize
    return word_tokenize(text, format="text")

df["Content_wt"] = df["Content"].parallel_apply(tokenize_text)

In [11]:
df[["Content", "Content_wt"]].head()

,Content,Content_wt
0,"Áo ba lỗ thun gân ,form body tôn dáng Áo Ba Lỗ...","Áo ba lỗ thun gân , form body tôn_dáng Áo_Ba_L..."
1,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu ...","Áo Ba_Lỗ Nam_Trắng Chất_Cotton Siêu_Mát , Siêu..."
2,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...","Áo Ba_Lỗ Nam_Tyasuo chất vải co_dãn mát , khôn..."
3,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON Áo Ba Lỗ Ch...,ÁO BA_LỖ HÀNG_VIỆT_NAM 100 % COTTON_Áo Ba_Lỗ C...
4,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Thun_Nam Thể_Thao Ba_Lỗ Mẫu_Mới Siêu_Đẹp ( ...


In [12]:
# remove stop words
# df["Content_wt"]=df["Content_wt"].apply(lambda x: ' '.join([word for word in x.split() if not word in stop_words]))

def remove_stopwords(text, stops=stop_words):
    """Function that passes stop_words as a parameter to avoid the NameError"""
    return ' '.join([word for word in text.split() if word not in stops])

#also use parallel
# this way need to define the func remove_stopwords becasue the stop_words variable above is not
#... automatically available in the worker processes
df["Content_wt"]=df["Content_wt"].parallel_apply(remove_stopwords)

In [13]:
# maybe dump into a pickle file so that no need to re-run in case of crash.

df.to_pickle('./df_content_wt.pkl')

In [14]:
# load pkl file if needed

# df = pd.read_pickle('./df_content_wt.pkl')

## Using Gensim

In [15]:
# Tokenize(split) the sentences into words
content_Gensim = [[text for text in x.split()] for x in df.Content_wt]

In [16]:
len(content_Gensim)

46828

In [17]:
content_Gensim[:1]

[['Áo',
  'lỗ',
  'thun',
  'gân',
  ',',
  'form',
  'body',
  'tôn_dáng',
  'Áo_Ba_Lỗ',
  'Chiều',
  'tay_áo',
  'Khác',
  'Phong_cách',
  'Thể_Cơ_bản',
  ',',
  'Đường_phố',
  ',',
  'Nhiệt_đới',
  'Tall_Fit',
  'Có',
  'Xuất_xứ',
  'Việt_Nam_Dáng',
  'kiểu',
  'áo',
  'Ôm_sát',
  'Chất_liệu',
  'Cotton_Mẫu',
  'Sọc_tăm',
  'gân',
  'nổi',
  'Kho',
  'hàng',
  '75317',
  'Gửi',
  'Bà_Rịa',
  'Vũng_Tàu',
  'BEST',
  'TANK_TOPS',
  'FOR_MEN',
  '2021',
  'Áo_Ba',
  'Lỗ_Quốc_Dân',
  'Cho_Nam',
  'Bảng_size',
  'Size_M',
  'Size_L',
  '55',
  'kg',
  'Size_XL',
  '70',
  'kg',
  'Size_XXL',
  '75',
  'kg',
  '80',
  'kg',
  '(',
  'Bảng_size',
  'chuẩn',
  'mặc',
  'kiểu',
  'body_nha',
  'quý_khách',
  'không',
  'thích',
  'mặc',
  'ôm',
  '1',
  'size',
  'mặc',
  'rộng',
  'rộng',
  '2',
  'size',
  'bảng',
  'size',
  'tính_chất',
  'tham_khảo',
  'khách_thể',
  'hình',
  'không',
  'cân_đối',
  '.',
  'Để',
  'tư_vấn',
  'chính_xác',
  'vui_lòng',
  'inbox',
  'shop',
  ')',
  'Kí

In [18]:
# data preprocessing for gensim
from utils import data_preprocessing_for_gensim

content_Gensim_processed = data_preprocessing_for_gensim(text=content_Gensim,remove_number=True, remove_special_chars= True, stop_words=stop_words)

In [19]:
content_Gensim_processed[:1]

[['áo',
  'lỗ',
  'thun',
  'gân',
  'form',
  'body',
  'tôn_dáng',
  'áo_ba_lỗ',
  'chiều',
  'tay_áo',
  'phong_cách',
  'thể_cơ_bản',
  'đường_phố',
  'nhiệt_đới',
  'tall_fit',
  'xuất_xứ',
  'việt_nam_dáng',
  'kiểu',
  'áo',
  'ôm_sát',
  'chất_liệu',
  'cotton_mẫu',
  'sọc_tăm',
  'gân',
  'nổi',
  'kho',
  'hàng',
  'gửi',
  'bà_rịa',
  'vũng_tàu',
  'best',
  'tank_tops',
  'for_men',
  'áo_ba',
  'lỗ_quốc_dân',
  'cho_nam',
  'bảng_size',
  'size_m',
  'size_l',
  'kg',
  'size_xl',
  'kg',
  'size_xxl',
  'kg',
  'kg',
  'bảng_size',
  'chuẩn',
  'mặc',
  'kiểu',
  'body_nha',
  'quý_khách',
  'không',
  'thích',
  'mặc',
  'ôm',
  'size',
  'mặc',
  'rộng',
  'rộng',
  'size',
  'bảng',
  'size',
  'tính_chất',
  'tham_khảo',
  'khách_thể',
  'hình',
  'không',
  'cân_đối',
  'tư_vấn',
  'chính_xác',
  'vui_lòng',
  'inbox',
  'shop',
  'kích_thước',
  'size',
  'cm',
  'rộng',
  'vòng',
  'ngực',
  'size',
  'rộng',
  'cm',
  'vòng',
  'ngực',
  'size_xl',
  'cm',
  'rộng

In [20]:
# Add the processed content to the DataFrame
df['content_processed'] = content_Gensim_processed

In [21]:
# Obtain the number of features based on dictionary: Use corpora.Dictionary
dictionary = corpora.Dictionary(df['content_processed'] )

In [22]:
print(dictionary)

Dictionary<144190 unique tokens: ['best', 'body', 'body_nha', 'bà_rịa', 'bảng']...>


In [23]:
# List of features in dictionary
dictionary.token2id

{'best': 0,
 'body': 1,
 'body_nha': 2,
 'bà_rịa': 3,
 'bảng': 4,
 'bảng_size': 5,
 'chiều': 6,
 'cho_nam': 7,
 'chuẩn': 8,
 'chính_xác': 9,
 'chất_liệu': 10,
 'cm': 11,
 'cotton_mẫu': 12,
 'cân_đối': 13,
 'cơ_thể': 14,
 'for_men': 15,
 'form': 16,
 'gân': 17,
 'gửi': 18,
 'hàng': 19,
 'hình': 20,
 'inbox': 21,
 'kg': 22,
 'kho': 23,
 'khách_thể': 24,
 'không': 25,
 'kiểu': 26,
 'kích_thước': 27,
 'l': 28,
 'lỗ': 29,
 'lỗ_quốc_dân': 30,
 'm': 31,
 'mặc': 32,
 'ngực': 33,
 'nhiệt_đới': 34,
 'nách': 35,
 'nổi': 36,
 'phong_cách': 37,
 'quý_khách': 38,
 'rộng': 39,
 'shop': 40,
 'size': 41,
 'size_l': 42,
 'size_m': 43,
 'size_xl': 44,
 'size_xxl': 45,
 'sát': 46,
 'sọc_tăm': 47,
 'tall_fit': 48,
 'tank_tops': 49,
 'tay_áo': 50,
 'tham_khảo': 51,
 'thun': 52,
 'thích': 53,
 'thể_cơ_bản': 54,
 'tính_chất': 55,
 'tôn_dáng': 56,
 'tư_vấn': 57,
 'việt_nam_dáng': 58,
 'vui_lòng': 59,
 'vòng': 60,
 'vũng_tàu': 61,
 'xuất_xứ': 62,
 'xxl': 63,
 'áo': 64,
 'áo_ba': 65,
 'áo_ba_lỗ': 66,
 'ôm': 67,


In [24]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)

In [25]:
feature_cnt

144190

In [26]:
# Obtain corpus based on dictionary (dense matrix)
# convert text to BoW (Bag of Words)
# doc2bow(text): This method converts a list of words (the text) into a bag-of-words (BoW) representation. In this representation:
# Each unique word in the document is represented by its corresponding integer ID from the dictionary.
# The output is a list of tuples, where each tuple contains the word ID and its frequency in the document. 
# For example, if a document contains the word "apple" twice and "banana" once, the output might look like [(0, 2), (1, 1)], 
# where 0 and 1 are the IDs for "apple" and "banana", respectively.

corpus = [dictionary.doc2bow(text) for text in df['content_processed']] 

In [27]:
corpus[0] # format is <id>, <frequency>

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 3),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 10),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 2),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 4),
 (23, 1),
 (24, 1),
 (25, 4),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 4),
 (33, 4),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 7),
 (40, 1),
 (41, 6),
 (42, 1),
 (43, 1),
 (44, 2),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 5),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 2),
 (65, 1),
 (66, 1),
 (67, 2),
 (68, 1),
 (69, 1)]

In [28]:
# Use TF-IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)


In [29]:

# calculate the similarity between the corpus and the index
# similarity_index_mx = similarities.SparseMatrixSimilarity(tfidf[corpus],
#                                             num_features = feature_cnt)

### this use too much memory, often crashs -> try to use LSI instead.

In [30]:
# to optimize the calculation of similarity, reduce the memory, using LSI (Latent Sematic Indexing)
# another option is using num_best parameter to keep only only top x most similar items for each document to reduce memory usage.

# Create LSI model
lsi_model = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=300)  # Reduce dimensions to 300 topics

# Transform corpus to LSI space
lsi_corpus = lsi_model[tfidf[corpus]]

In [31]:
# Create similarity index in the reduced space
similarity_index = similarities.MatrixSimilarity(lsi_corpus, num_features=300)

######## gensim_df = pd.DataFrame(similarity_index)
* Converting the entire similarity matrix to a DataFrame is problematic because:
* dataset has 46,828 documents (as shown in cell 17)
* This creates a 46,828 × 46,828 matrix (over 2 billion elements)
* Even with optimized storage, this requires gigabytes of memory

instead, do this:
* Keep the similarity index as is (it's optimized for memory usage)
* Calculate similarities only when needed for specific products
* Only retrieve the top N similar items for a given product


In [32]:

# Don't convert the entire similarity index to a DataFrame
# gensim_df = pd.DataFrame(similarity_index)

# create a function to get recommendations using the similarity index directly
# moved it to utils


In [33]:
# Import the reload function
from importlib import reload
import utils

# Reload the utils module to get the latest changes
# by this I don't need to restart the kernel everytime I modify the function in Utils
reload(utils)

# Now try your function call again
from utils import get_recommendations_gensim

In [34]:
pd.set_option('display.max_colwidth', None)

### Use case 1: user select a product (productid as input)

In [35]:
# display the product 19656
selected_product = 19656
selected_product_details= df[df['product_id'] == 19656]

In [36]:
selected_product_details

,product_id,product_name,category,sub_category,link,image,price,rating,description,description_clean,Content,Content_wt,content_processed
1156,19656,(A393) DỆT KIM ĐÔNG XUÂN ÁO MAY Ô NAM 3 LỖ MÀU TRẮNG/ ĐEN 209-A0393 DÁNG BODY ÔM SLIM FIT,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/(A393)-D%E1%BB%86T-KIM-%C4%90%C3%94NG-XU%C3%82N-%C3%81O-MAY-%C3%94-NAM-3-L%E1%BB%96-M%C3%80U-TR%E1%BA%AENG-%C4%90EN-209-A0393-D%C3%81NG-BODY-%C3%94M-SLIM-FIT-i.279296612.2921494470?sp_atk=16e2bad5-832a-41e6-9ffa-49491761aa4f,NaN,72000.0,5.0,"Áo Ba Lỗ Kho hàng 223 Gửi từ Huyện Đông Anh, Hà Nội **** DÁNG ÔM HÀNG PHÂN PHỐI TRỰC TIẾP CỦA CÔNG TY DỆT KIM ĐÔNG XUÂN CAM KẾT HÀNG CHÍNH HÃNG CHẤT LIỆU 100%, THOÁNG MÁT VÀ THẤM HÚT MỒ HÔI, KHÔNG NHƯ HÀNG CHỢ PHA NHIỀU NYLON Mỗi sản phẩm đều được để trong túi riêng, in đầy đủ thông tin Với chất liệu 100% cotton, mặt vải mịn, thoáng khí, co giãn tốt, có khả năng thấm hút nước cao, thoát nước nhanh khô, đem lại cảm giác thoải mái, tự tin cho người sử dụng Quy trình xử lý vải an toàn, không độc hại, không gây kích ứng da, theo tiêu chuẩn Nhật Bản Áo may ô A0393 là 1 sự lựa chọn hoàn hảo cho nam giới. #aobalonam #aomayonam #aolotnam #aobalonamdetkimdongxuan #aomayonamdetkimdongxuan #aolotnamdetkimdongxuan #aolotnamtrang #aobalonamtrang #aomayonamtrang #aomayonamcotton #aomayonamcotton #aothunnam #aothunnammautrang #aothunnamtrang #aothunnamtrangcoctay #aothunnamtrangngantay","Áo Ba Lỗ Kho hàng 223 Gửi từ Huyện Đông Hà Nội DÁNG ÔM HÀNG PHÂN PHỐI TRỰC TIẾP CỦA CÔNG TY DỆT KIM ĐÔNG XUÂN CAM KẾT HÀNG CHÍNH HÃNG CHẤT LIỆU THOÁNG MÁT VÀ THẤM HÚT MỒ HÔI, KHÔNG NHƯ HÀNG CHỢ PHA NHIỀU NYLON Mỗi sản phẩm đều được để trong túi riêng, in đầy đủ thông tin Với chất liệu 100 mặt vải mịn, thoáng khí, co giãn tốt, có khả năng thấm hút nước thoát nước nhanh khô, đem lại cảm giác thoải mái, tự tin cho người sử dụng Quy trình xử lý vải an toàn, không độc hại, không gây kích ứng theo tiêu chuẩn Nhật Bản Áo may ô A0393 là 1 sự lựa chọn hoàn hảo cho nam giới. aobalonam aomayonam aolotnam aobalonamdetkimdongxuan aomayonamdetkimdongxuan aolotnamdetkimdongxuan aolotnamtrang aobalonamtrang aomayonamtrang aomayonamcotton aomayonamcotton aothunnam aothunnammautrang aothunnamtrang aothunnamtrangcoctay aothunnamtrangngantay","(A393) DỆT KIM ĐÔNG XUÂN ÁO MAY Ô NAM 3 LỖ MÀU TRẮNG/ ĐEN 209-A0393 DÁNG BODY ÔM SLIM FIT Áo Ba Lỗ Kho hàng 223 Gửi từ Huyện Đông Hà Nội DÁNG ÔM HÀNG PHÂN PHỐI TRỰC TIẾP CỦA CÔNG TY DỆT KIM ĐÔNG XUÂN CAM KẾT HÀNG CHÍNH HÃNG CHẤT LIỆU THOÁNG MÁT VÀ THẤM HÚT MỒ HÔI, KHÔNG NHƯ HÀNG CHỢ PHA NHIỀU NYLON Mỗi sản phẩm đều được để trong túi riêng, in đầy đủ thông tin Với chất liệu 100 mặt vải mịn, thoáng khí, co giãn tốt, có khả năng thấm hút nước thoát nước nhanh khô, đem lại cảm giác thoải mái, tự tin cho người sử dụng Quy trình xử lý vải an toàn, không độc hại, không gây kích ứng theo tiêu chuẩn Nhật Bản Áo may ô A0393 là 1 sự lựa chọn hoàn hảo cho nam giới. aobalonam aomayonam aolotnam aobalonamdetkimdongxuan aomayonamdetkimdongxuan aolotnamdetkimdongxuan aolotnamtrang aobalonamtrang aomayonamtrang aomayonamcotton aomayonamcotton aothunnam aothunnammautrang aothunnamtrang aothunnamtrangcoctay aothunnamtrangngantay","( A393 ) DỆT_KIM_ĐÔNG XUÂN_ÁO MAY_Ô_NAM 3 LỖ MÀU TRẮNG / ĐEN 209 - A0393_DÁNG BODY_ÔM SLIM FIT_Áo Ba_Lỗ Kho hàng 223 Gửi Huyện_Đông Hà_Nội DÁNG_ÔM HÀNG_PHÂN_PHỐI TRỰC_TIẾP CỦA CÔNG_TY DỆT_KIM_ĐÔNG XUÂN_CAM_KẾT HÀNG_CHÍNH HÃNG CHẤT_LIỆU THOÁNG_MÁT VÀ THẤM_HÚT MỒ_HÔI , KHÔNG NHƯ_HÀNG_CHỢ PHA_NHIỀU NYLON Mỗi sản_phẩm túi , in đầy_đủ thông_tin Với chất_liệu 100 mặt vải mịn , thoáng_khí , co_giãn tốt , khả_năng thấm_hút thoát khô , đem cảm_giác thoải_mái , tự_tin Quy_trình vải an_toàn , không độc_hại , không kích_ứng tiêu_chuẩn Nhật_Bản Áo may_ô A0393 1 lựa_chọn hoàn_hảo nam_giới . aobalonam aomayonam aolotnam aobalonamdetkimdongxuan aomayonamdetkimdongxuan aolotnamdetkimdongxuan aolotnamtrang aobalonamtrang aomayonamtrang aomayonamcotton aomayonamcotton aothunnam aothunnammautrang aothunnamtrang aothunnamtrangcoctay aothu

In [37]:
# example, use view product_id = 19656
gensim_prod_recom = get_recommendations_gensim(similarity_index=similarity_index,df=df,tfidf=tfidf,lsi_model=lsi_model,dictionary=dictionary,product_id=selected_product,nums=10)

In [38]:
gensim_prod_recom

,product_id,product_name,sub_category,rating,similarity_score
2192,20692,Áo may ô Dệt Kim Đông Xuân 154 A0396,"Áo Hoodie, Áo Len & Áo Nỉ",5.0,0.750625
1177,19677,"Áo ba lỗ cotton nam hàng xuất khẩu, áo ba lỗ mặc nhà màu trắng hàng đẹp",Áo Ba Lỗ,5.0,0.700594
93,1993,Áo Ba lỗ nam 100% coton - hãng Ledatex (Màu xám) có big size,Áo Ba Lỗ,4.9,0.676790
36,1936,"Áo Ba Lỗ Nam Màu Trắng⭐⭐ JUVENO⭐⭐, co giãn 4 chiều, đủ size M-XXL - RSC01",Áo Ba Lỗ,5.0,0.655181
804,19304,"Áo ba lỗ, áo ba lỗ nam cao cấp - Hàng Việt nam chất lượng cao 100% cotton, mặc mát vào mùa hè, ấm vào mùa đông ..",Áo Ba Lỗ,5.0,0.590434
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,4.8,0.590108
661,191461,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Áo Ba Lỗ,0.0,0.588449
810,19310,Áo Ba Lỗ Nam Thoáng Mát ABL01,Áo Ba Lỗ,4.4,0.572474
824,19324,Áo ba lỗ cotton nam hàng loại 2,Áo Ba Lỗ,4.7,0.569431
1323,19823,Áo Ba Lỗ YODY Áo Lót Nam Vải Cotton Cao Cấp BLM3001,Áo Ba Lỗ,5.0,0.557504


### Use case 2: user search for using some keywords (text input)


In [39]:
### Use case 2: user search using keywords, sentence
queryString = "Áo ba lỗ cotton nam"
gensim_recommendations_by_query = get_recommendations_gensim(similarity_index=similarity_index,df=df,tfidf=tfidf,lsi_model=lsi_model,dictionary=dictionary,query=queryString,nums=10,stop_words=stop_words)

In [40]:
gensim_recommendations_by_query

,product_id,product_name,sub_category,rating,similarity_score
419,191219,Áo ba lỗ nam TTT2222,Áo Ba Lỗ,0.0,0.883626
612,191412,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ và tanktop, Áo ba lỗ vnxk [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.844249
705,19205,Áo ba lỗ nam sát nách form rộng vải thun cotton mềm mịn thoáng mát unisex big size BIBINO68 Áo tank top nam nữ cao cấp,Áo Ba Lỗ,4.7,0.844222
1091,19591,Combo 3 Áo ba lỗ nam áo cộc tay nam,Áo Ba Lỗ,5.0,0.837023
557,191357,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ phom rộng, Áo ba lỗ hàng hiệu [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.836305
509,191309,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ mùa hè, Áo ba lỗ nam cổ tròn [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.836187
561,191361,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ mùa hè, Áo ba lỗ nam thể thao [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.834444
559,191359,"[Chuyên Sỉ Áo Thun] Áo ba lỗ sát nách nam, Áo ba lỗ nam phối màu",Áo Ba Lỗ,0.0,0.829950
465,191265,Áo ba lỗ đen họa tiết năng động,Áo Ba Lỗ,5.0,0.827171
532,191332,"Xưởng May Chuyên Sỉ ✨ [Áo 3 lỗ cotton thời trang] Áo thun ba lỗ nam giá rẻ, Áo ba lỗ vnxk [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.824486


### Using Cosine

In [41]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])

In [43]:
# # Tính toán độ tương đồng
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [44]:
# df_show = pd.DataFrame(cosine_sim)
# df_show

# this will crash the machine as the dataframe is huge.

In [45]:
reload(utils)
from utils import get_recommendations_cosine

In [46]:
cosine_recommendations_by_query = get_recommendations_cosine(tfidf_matrix=tfidf_matrix, df=df,product_id=selected_product,nums=10,vectorizer=vectorizer)

In [47]:
cosine_recommendations_by_query

,product_id,product_name,sub_category,rating,similarity_score
1265,19765,Áo Ba Lỗ Nam Thể Thao Hyun Boutique - WH01C - Hyun Boutique,Áo Ba Lỗ,5.0,0.692265
618,191418,COMBO - 4 áo thun nam - 1 vớ Hyun Boutique - Áo ba lỗ nam Cotton thoáng mát,Áo Ba Lỗ,0.0,0.573018
158,19158,Áo Ba Lỗ Tank Top Mau Khô Không Đường May - 2137-KHOTONG,Áo Ba Lỗ,4.7,0.247861
926,19426,"Áo Ba Lỗ Tank Top Mau Khô Không Đường May, không nhăn",Áo Ba Lỗ,4.0,0.247479
1311,19811,Áo Ba Lỗ Tank Top Mau Khô Không Đường May 2137 shop88,Áo Ba Lỗ,5.0,0.246448
1193,19693,Áo Ba Lỗ Tank Top Mau Khô Không Đường May - 2137_Miễn Phí Ship,Áo Ba Lỗ,5.0,0.244475
646,191446,"[Áo 3 lỗ cotton thời trang] Áo ba lỗ nam nữ, Áo ba lỗ cho nam [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.197422
529,191329,"(Giá Tại Kho) [Áo 3 lỗ nam đẹp] Áo thun 3 lỗ nam body, Áo thun 3 lỗ nam nách rộng [Áo ba lỗ thể thao in hình chất lượng",Áo Ba Lỗ,0.0,0.192640
513,191313,"Thời Trang Cao Cấp Giá Sỉ ✨ [Áo 3 lỗ nam đẹp] Áo ba lỗ mùa hè cho nam, Áo ba lỗ nam tập gym [Áo ba lỗ thể thao in hình c",Áo Ba Lỗ,0.0,0.192062
643,191443,"[Áo 3 lỗ cotton thời trang] Áo ba lỗ hàng hiệu, Áo ba lỗ body nam [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.190547


In [48]:
example_query_recom = "Áo ba lỗ cotton nam"
cosine_query_recom = get_recommendations_cosine(tfidf_matrix=tfidf_matrix, df=df,query=example_query_recom,nums=10,vectorizer=vectorizer)

In [49]:
cosine_query_recom

,product_id,product_name,sub_category,rating,similarity_score
561,191361,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ mùa hè, Áo ba lỗ nam thể thao [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.677885
509,191309,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ mùa hè, Áo ba lỗ nam cổ tròn [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.674268
532,191332,"Xưởng May Chuyên Sỉ ✨ [Áo 3 lỗ cotton thời trang] Áo thun ba lỗ nam giá rẻ, Áo ba lỗ vnxk [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.665093
612,191412,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ và tanktop, Áo ba lỗ vnxk [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.650462
557,191357,"(Giá Sỉ) [Áo 3 lỗ cotton thời trang] Áo ba lỗ phom rộng, Áo ba lỗ hàng hiệu [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.639389
533,191333,"Xưởng May Chuyên Sỉ ✨ [Áo 3 lỗ cotton thời trang] Áo 3 lỗ in hình siêu nét, Áo 3 lỗ nam 3d chất vải mềm mịn [Áo ba lỗ in",Áo Ba Lỗ,0.0,0.637576
562,191362,"[Chuyên Sỉ Áo Thun] Áo ba lỗ in họa tiết siêu đẹp, Áo ba lỗ nam cổ tròn",Áo Ba Lỗ,0.0,0.636339
646,191446,"[Áo 3 lỗ cotton thời trang] Áo ba lỗ nam nữ, Áo ba lỗ cho nam [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.633394
534,191334,"Xưởng May Chuyên Sỉ ✨ [Áo 3 lỗ cotton thời trang] Áo thun ba lỗ, Áo ba lỗ thun lạnh [Áo ba lỗ in họa tiết cao cấp]",Áo Ba Lỗ,0.0,0.633246
611,191411,"➡️ [Áo 3 lỗ cotton thời trang] Áo thun ba lỗ kiểu nam, Áo thun ba lỗ nam giá sỉ [Áo ba lỗ in họa tiết cao cấp] [Siêu Đẹp",Áo Ba Lỗ,0.0,0.628105


In [51]:
# saving models for Streamlit.
# save Gensime
import pickle
import os

# Create directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('data', exist_ok=True)

# Save LSI model
with open('models/lsi_model.pkl', 'wb') as f:
    pickle.dump(lsi_model, f)

# Save TF-IDF model
with open('models/tfidf_model.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Save dictionary
with open('models/dictionary.pkl', 'wb') as f:
    pickle.dump(dictionary, f)

# Save similarity index
with open('models/similarity_index.pkl', 'wb') as f:
    pickle.dump(similarity_index, f)

# Save processed data
content_processed = df['content_processed'].tolist()  # List of tokenized words
Content_wt = df['Content_wt'].tolist()  # List of processed text strings

# Then save them along with your DataFrame
with open('data/processed_data.pkl', 'wb') as f:
    pickle.dump({
        'df': df,
        'content_processed': content_processed,
        'Content_wt': Content_wt
    }, f)